# `with` en Python — principes et exemples (fichiers, `tf.GradientTape`)

## Objectif
Présenter ce que fait `with`, pourquoi l’utiliser, et illustrer par des exemples concrets avec des **fichiers** puis avec **TensorFlow**.

## Idée centrale
`with` ouvre/démarre une ressource au début du bloc, puis **la libère ou l’arrête automatiquement** à la fin du bloc — y compris si l’exécution quitte le bloc prématurément.

## Pourquoi l’utiliser ?
- **Robustesse** : libération fiable des ressources.
- **Lisibilité** : évite le code « plomberie » (`close()`, etc.).
- **Concision** : se substitue à un `try/finally` explicite.

## Syntaxe générale
```python
with objet_contexte as alias:
    # utiliser alias dans le bloc
```
`alias` est la valeur fournie par l’objet via son protocole d’entrée dans le contexte.


## 1) Exemple de base : fichiers
Ouvrir, utiliser, puis fermer un fichier automatiquement.


In [ ]:
# Écriture : fermeture automatique en sortie de bloc
with open("demo.txt", "w", encoding="utf-8") as f:  # f est l'alias
    f.write("Bonjour, Python avec with.\n")

# Lecture : idem
with open("demo.txt", "r", encoding="utf-8") as f:
    contenu = f.read()
print(contenu.strip())

### Plusieurs ressources dans un même `with`
Il est possible de gérer plusieurs contextes simultanément.


In [ ]:
# Préparation d'une source
with open("source.txt", "w", encoding="utf-8") as src:
    src.write("Ligne A\nLigne B\n")

# Copie de source.txt vers copie.txt
with open("source.txt", "r", encoding="utf-8") as src, \
     open("copie.txt", "w", encoding="utf-8") as dst:
    for ligne in src:
        dst.write(ligne)

# Vérification
with open("copie.txt", "r", encoding="utf-8") as f:
    print("copie.txt ->")
    print(f.read().strip())

## 2) `with` et TensorFlow : `tf.GradientTape`

**Terminologie.** Le mot *tape* dans `GradientTape` se traduit utilement par **« enregistreur de gradients »** (métaphore d’une **bande d’enregistrement**). Entrer dans le bloc `with` démarre l’enregistrement des opérations appliquées aux tenseurs « surveillés » ; sortir du bloc l’arrête. On peut ensuite demander les **gradients** de sorties par rapport aux variables surveillées.

*Pré-requis :* `pip install tensorflow`


In [ ]:
import tensorflow as tf

# Variable "entraînable" (surveillée automatiquement par la tape)
x = tf.Variable(3.0)

with tf.GradientTape() as tape:        # démarrer l'enregistreur de gradients
    y = x**2 + 2*x + 1                  # y = (x + 1)^2
# sortie du bloc -> arrêt de l'enregistrement

dy_dx = tape.gradient(y, x)             # dy/dx = 2*x + 2 en x=3 -> 8.0
print("dy/dx =", float(dy_dx))

### Tape persistante (plusieurs gradients)
Par défaut, une tape calcule les gradients **une seule fois**. Avec `persistent=True`, on peut en demander plusieurs, puis libérer explicitement la tape.


In [ ]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:
    y1 = x**2        # dy1/dx = 2x -> 4.0
    y2 = 3*x + 5     # dy2/dx = 3   -> 3.0

g1 = tape.gradient(y1, x)
g2 = tape.gradient(y2, x)
del tape  # bonne pratique : libérer l'enregistreur persistant

print("dy1/dx =", float(g1), "| dy2/dx =", float(g2))

### Surveiller un `tf.constant`
Les `tf.Variable` sont surveillées automatiquement. Pour un `tf.constant`, il faut le **déclarer** à l’enregistreur.


In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:
    tape.watch(x)     # important avec un constant
    y = x**2
print("d(x^2)/dx|_{x=3} =", float(tape.gradient(y, x)))

---
## À retenir
- `with ... as alias:` ouvre un **contexte** puis le **ferme automatiquement** en sortie.
- Cas d’usage immédiat : **fichiers** (ouvrir/fermer sans `close()` explicite).
- Avec `tf.GradientTape`, le bloc `with` joue le rôle d’un **enregistreur de gradients** : en entrée on commence à enregistrer, en sortie on arrête, puis on demande les gradients.
- Pour plusieurs calculs à partir de la même trace : `persistent=True`, puis libération explicite.
